In [1]:
import folium
import geopandas as gpd
import pandas as pd

In [2]:
# Import the shapefile with data
gdf = gpd.read_file('data/Census_Tracts_2020_Redistricting_Data.shp')
# Import the labels that I gave to the tracts 
labels = pd.read_csv('data/tract_labels.csv').drop(['TRACT_NO', 'FIPS2020'], axis=1)
# labels = pd.read_csv('data/tract_labels and validation - details.csv').drop(['TRACT_NO', 'FIPS2020'], axis=1)
# Cast types to enable merge
gdf['STATE'] = gdf['STATE'].astype('int64')
gdf['COUNTY'] = gdf['COUNTY'].astype('int64')
gdf['TRACT'] = gdf['TRACT'].astype('int64')
# Merge the shapefile with the labels
gdf = gdf.merge(labels, on=['STATE', 'COUNTY', 'TRACT'], how='left')
gdf = gdf[gdf['LABEL']!='Not Portland']
gdf = gdf[gdf['LABEL'].notnull()]
gdf[gdf['LABEL'].notnull()].head(2)

,OBJECTID,STATE,COUNTY,TRACT,TRACT_NO,FIPS2020,P2_001N,P2_002N,P2_005N,P2_006N,...,Shape__Are,Shape__Len,geometry,POPULATION,LABEL,LABEL_NOTE,NEIGHBORHOOD_ASSOCIATION,OPTION_1,OPTION_1_NAME,Unnamed: 11
210,211,41,51,2401,24.01,41051002401,2794,173,2096,175,...,9.064165e+06,12542.520021,"POLYGON ((-122.65860 45.54827, -122.65768 45.5...",2794.0,NE,inner NE,NECN,3.0,3-NECN-CNN,3.0
211,212,41,51,2402,24.02,41051002402,3830,308,2905,180,...,9.703143e+06,13945.775924,"POLYGON ((-122.65760 45.53435, -122.65760 45.5...",3830.0,NE,inner NE,NECN,3.0,3-NECN-CNN,3.0


Geopandas helper maps

In [3]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Population
# ax = gdf.plot(figsize=(100, 100), 
#                  column='P2_001N', #'P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['P2_001N'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])

In [4]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Census Tract
# ax = gdf.plot(figsize=(50, 50), 
#                  column='P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['TRACT_NO'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])

Folium maps for some options

In [5]:
# Create a map and add choropleth by population for each tract
def create_map(gdf):
    m = folium.Map(location=[45.5, -122.5], 
                   zoom_start=10, 
                   width='100%')
    cp = folium.Choropleth(
        geo_data=gdf,
        name='Mulntomah County Population by Census Tract',
        data=gdf,
        columns=['TRACT_NO', 'P2_001N'],
        key_on="feature.properties.TRACT_NO",
        fill_color="OrRd", # yellow green blue : YlGnBu
        #bins=[0, 300, 600, 900, 2000, 3000],
        label=['P2_001N'],
        tooltip=['TRACT_NO', 'P2_001N'],
        fill_opacity=0.5,
        line_opacity=0.9,
        legend_name='Population per Census Tract',
    ).add_to(m)

    # This adds tooltips:  https://medium.com/analytics-vidhya/interactive-choropleth-map-in-python-using-folium-4e1479d9e568
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    tooltipadd = folium.features.GeoJson(
        gdf,
        style_function=style_function, 
        control=False,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['TRACT_NO', 'P2_001N'],
            aliases=['TRACT_NO', 'P2_001N'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(tooltipadd)
    m.keep_in_front(tooltipadd)
    folium.LayerControl().add_to(m)
    return m


In [6]:
# create_map(gdf)

Validations that the data is correctly labeled for quadrants

In [7]:
# # Filter what goes into the map 
# gdf_north = gdf[gdf['LABEL']=='N']
# create_map(gdf_north)

In [8]:
# # Filter what goes into the map 
# gdf_ne = gdf[gdf['LABEL']=='NE']
# create_map(gdf_ne)

In [9]:
# # Filter what goes into the map 
# gdf_se = gdf[gdf['LABEL']=='SE']
# create_map(gdf_se)

In [10]:
# # Filter what goes into the map 
# gdf_nw = gdf[gdf['LABEL']=='NW']
# create_map(gdf_nw)

In [11]:
# # Filter what goes into the map 
# gdf_sw = gdf[gdf['LABEL']=='SW']
# create_map(gdf_sw)

Validations that the data is correctly labeled for neighborhood associations

In [12]:
gdf['NEIGHBORHOOD_ASSOCIATION'].unique()
# # Filter what goes into the map 
# gdf_NECN = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='NECN']
# create_map(gdf_NECN)
# # Filter what goes into the map 
# gdf_SWNI = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='SWNI']
# create_map(gdf_SWNI)
# # Filter what goes into the map 
# gdf_NWNW = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='NWNW']
# create_map(gdf_NWNW)
# # Filter what goes into the map 
# gdf_EPNO = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='EPNO']
# create_map(gdf_EPNO)
# Filter what goes into the map 
# gdf_NPNS = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='NPNS']
# create_map(gdf_NPNS)
# # Filter what goes into the map 
# gdf_SEUL = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='SEUL']
# create_map(gdf_SEUL)
# # Filter what goes into the map 
# gdf_CNN = gdf[gdf['NEIGHBORHOOD_ASSOCIATION']=='CNN']
# create_map(gdf_CNN)

array(['NECN', 'SWNI', 'NWNW', 'EPNO', 'NPNS', 'SEUL', 'CNN', 'LLOYD'],
      dtype=object)

In [13]:
# Option 1 - Trying to keep the neighborhood associations whole, not 100% possible
option_1 = gdf[gdf['OPTION_1'].notnull()]
print(len(option_1))
option_1.head()

160


,OBJECTID,STATE,COUNTY,TRACT,TRACT_NO,FIPS2020,P2_001N,P2_002N,P2_005N,P2_006N,...,Shape__Are,Shape__Len,geometry,POPULATION,LABEL,LABEL_NOTE,NEIGHBORHOOD_ASSOCIATION,OPTION_1,OPTION_1_NAME,Unnamed: 11
210,211,41,51,2401,24.01,41051002401,2794,173,2096,175,...,9.064165e+06,12542.520021,"POLYGON ((-122.65860 45.54827, -122.65768 45.5...",2794.0,NE,inner NE,NECN,3.0,3-NECN-CNN,3.0
211,212,41,51,2402,24.02,41051002402,3830,308,2905,180,...,9.703143e+06,13945.775924,"POLYGON ((-122.65760 45.53435, -122.65760 45.5...",3830.0,NE,inner NE,NECN,3.0,3-NECN-CNN,3.0
212,213,41,51,2501,25.01,41051002501,4672,233,3906,75,...,1.440436e+07,15280.345406,"POLYGON ((-122.64734 45.54051, -122.64733 45.5...",4672.0,NE,near hollywood,NECN,3.0,3-NECN-CNN,3.0
213,214,41,51,2502,25.02,41051002502,4614,363,3462,163,...,1.197719e+07,14165.618088,"POLYGON ((-122.64735 45.53898, -122.64633 45.5...",4614.0,NE,near hollywood,NECN,3.0,3-NECN-CNN,3.0
214,215,41,51,6502,65.02,41051006502,4419,378,3383,113,...,2.069157e+07,23035.074700,"POLYGON ((-122.72498 45.46074, -122.72491 45.4...",4419.0,SW,near hills,SWNI,4.0,4-W-N-L,4.0


In [17]:
# Create a map and add choropleth for Option 1
m = folium.Map(location=[45.5, -122.5], 
               zoom_start=10, 
               width='100%')
folium.Choropleth(
    geo_data=option_1,
    name='Districts Option 1',
    data=option_1,
    columns=['TRACT_NO', 'OPTION_1'], 
    key_on='feature.properties.TRACT_NO',
    fill_color= 'Accent', # https://matplotlib.org/2.0.2/users/colormaps.html
    fill_opacity=0.5,
    line_opacity=0.9,
    legend_name='Districts Option 1',
).add_to(m)

In [16]:
# m.save('option_1.html')